In [ ]:
#pip install requests beautifulsoup4 PyPDF2 pdfplumber
import requests
from bs4 import BeautifulSoup
#import PyPDF2
import io
from urllib.parse import urljoin
from urllib.parse import urlparse, urljoin
import re
import logging
import pdfplumber


In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
def download_pdf(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return io.BytesIO(response.content)
    except requests.RequestException as e:
        logging.error(f"Error downloading PDF from {url}: {e}")
    return None

def find_pdf_links(url, domain):
    pdf_links = []
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            for link in soup.find_all('a', href=True):
                href = link['href']
                if 'mailto:' in href or not href.lower().endswith('.pdf'):
                    continue
                pdf_link = urljoin(domain, href)
                if domain in urlparse(pdf_link).netloc:
                    pdf_links.append(pdf_link)
    except requests.RequestException as e:
        logging.error(f"Error finding PDF links on {url}: {e}")
    return pdf_links

In [ ]:
# Function to search keywords in webpage content


def search_keywords_in_webpage(html, keywords):
    try:
        # Convert the HTML to lowercase for case-insensitive searching
        lower_html = html.lower()
        for keyword in keywords:
            # Use regular expression for precise word boundary matching
            if re.search(r'\b' + re.escape(keyword.lower()) + r'\b', lower_html):
                return True
    except Exception as e:
        logging.error(f"Error searching keywords in webpage: {e}")
    return False


def search_keywords_in_pdf(pdf_io, keywords):
    try:
        with pdfplumber.open(pdf_io) as pdf:
            for page in pdf.pages:
                text = page.extract_text()
                for keyword in keywords:
                    if keyword.lower() in text.lower():
                        return True
    except Exception as e:
        logging.error(f"Error searching keywords in PDF: {e}")
    return False




# Function to crawl website
def crawl_website(start_url, domain, keywords, max_depth=0):
    visited_urls = set()
    urls_to_visit = {start_url}
    found_keywords_summary = {}  # Dictionary to store summary of findings

    while urls_to_visit:
        current_url = urls_to_visit.pop()
        if current_url in visited_urls:
            continue

        visited_urls.add(current_url)

        # Check if the URL belongs to the original domain
        if urlparse(current_url).netloc != urlparse(domain).netloc:
            continue

        try:
            response = requests.get(current_url)
            if response.status_code == 200:
                logging.info(f"Crawling URL: {current_url}")

                # Search for keywords in the webpage
                if search_keywords_in_webpage(response.text, keywords):
                    logging.info(f"Keyword hit found at {current_url}")
                    found_keywords_summary[current_url] = keywords

                soup = BeautifulSoup(response.text, 'html.parser')
                for link in soup.find_all('a', href=True):
                    absolute_link = urljoin(domain, link['href'])

                    # Check if the absolute link is within the original domain
                    if urlparse(absolute_link).netloc == urlparse(domain).netloc:
                        if absolute_link not in visited_urls:
                            urls_to_visit.add(absolute_link)

            # Process PDF links after processing all anchor tags
            pdf_links = find_pdf_links(current_url, domain)           
            for pdf_link in pdf_links:
                pdf_io = download_pdf(pdf_link)
                if pdf_io and search_keywords_in_webpage(pdf_io, keywords):
                    logging.info(f"Found keywords in PDF: {pdf_link}")
                    found_keywords_summary[pdf_link] = keywords

        except requests.RequestException as e:
            logging.error(f"Error crawling {current_url}: {e}")

    return found_keywords_summary

In [ ]:
if __name__ == "__main__":
    domain = 'https://www.transitionpathwayinitiative.org/publications'
    start_url = domain
    keywords = ['sovereign']
    summary = crawl_website(start_url, domain, keywords)

    print("\nSummary of keyword findings:")
    for location, found_keywords in summary.items():
        print(f"Keywords found at {location}: {found_keywords}")


In [ ]:
def crawl_website(start_url, domain, keywords, max_depth=0):
    visited_urls = set()
    urls_to_visit = {start_url}
    found_keywords_summary = {}  # Dictionary to store summary of findings

    while urls_to_visit:
        current_url = urls_to_visit.pop()
        if current_url in visited_urls:
            continue

        if 'mailto:' in current_url:
            continue

        logging.info(f"Crawling: {current_url}")
        try:
            response = requests.get(current_url)
            if response.status_code == 200:
                if search_keywords_in_webpage(response.text, keywords):
                    found_keywords_summary[current_url] = keywords

                soup = BeautifulSoup(response.text, 'html.parser')
                for link in soup.find_all('a', href=True):
                    absolute_link = urljoin(domain, link['href'])
                    if absolute_link not in visited_urls:
                        urls_to_visit.add(absolute_link)

                pdf_links = find_pdf_links(current_url, domain)
                for pdf_link in pdf_links:
                    pdf_io = download_pdf(pdf_link)
                    if pdf_io and search_keywords_in_webpage(pdf_io, keywords):
                        logging.info(f"Found keywords in PDF: {pdf_link}")
                        found_keywords_summary[pdf_link] = keywords

            visited_urls.add(current_url)

        except requests.RequestException as e:
            logging.error(f"Error crawling {current_url}: {e}")
            visited_urls.add(current_url)

    return found_keywords_summary

In [ ]:
#BIN##


In [ ]:
# Define your parameters
domain = 'https://joliennoels.com'
start_url = domain
keywords = ['greenhouse gas', 'net zero']

# Start crawling
crawl_website(start_url, domain, keywords)

In [ ]:
def crawl_website(start_url, domain, keywords):
    visited_urls = set()
    urls_to_visit = {start_url}
    domain_name = urlparse(domain).netloc

    while urls_to_visit:
        current_url = urls_to_visit.pop()
        if current_url in visited_urls:
            continue

        # Check if the URL belongs to the same domain
        if urlparse(current_url).netloc != domain_name:
            continue

        print(f"Crawling: {current_url}")
        
        try:
            response = requests.get(current_url)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                for link in soup.find_all('a', href=True):
                    # Complete URL
                    absolute_link = urljoin(current_url, link['href'])

                    # Skip if it's an email link or outside of domain
                    if 'mailto:' in absolute_link or urlparse(absolute_link).netloc != domain_name:
                        continue

                    if absolute_link not in visited_urls:
                        urls_to_visit.add(absolute_link)

                # Here, you would continue with processing PDF links as before
                # ...

        except requests.RequestException:
            pass

        visited_urls.add(current_url)

In [ ]:
def crawl_website(start_url, domain, keywords):
    visited_urls = set()
    urls_to_visit = {start_url}

    while urls_to_visit:
        current_url = urls_to_visit.pop()
        if current_url in visited_urls:
            continue

        if 'mailto:' in current_url:
            continue

        print(f"Crawling: {current_url}")

        pdf_links = find_pdf_links(current_url, domain)
        for pdf_link in pdf_links:
            pdf_io = download_pdf(pdf_link)
            if pdf_io and search_keywords_in_pdf(pdf_io, keywords):
                print(f"Found keywords in PDF: {pdf_link}")

        visited_urls.add(current_url)

        # Add new URLs to the queue (basic version, no check for same domain)
        soup = BeautifulSoup(requests.get(current_url).text, 'html.parser')
        for link in soup.find_all('a', href=True):
            absolute_link = urljoin(domain, link['href'])
            if absolute_link not in visited_urls:
                urls_to_visit.add(absolute_link)